# Testing with Concept Activation Vectors (TCAV)

Introduced by [Kim et al. (2018)](https://arxiv.org/pdf/1711.11279.pdf)

## 1) prepare dataset

We need to:

1. Create concepts of interest defined by input images
2. Create random concepts
3. Define class of interest

In [14]:
concept = "striped"

In [13]:
from torchvision import transforms

preprocessing = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [33]:
from pathlib import Path
from PIL import Image

import torch

def get_images_input(images_path, transform):
    imgs_files = list(Path(images_path).iterdir())
    
    prepro_imgs = []
    for file in imgs_files:
        img = Image.open(file).convert("RGB")
        img_prepro = transform(img)
        img_unsq = img_prepro.unsqueeze(0)
        prepro_imgs.append(img_unsq)
    
    imgs_tensor = torch.cat(prepro_imgs)
    
    return imgs_tensor
    

In [37]:
concept_images = get_images_input(f"/Users/martina.gonzales/data/tcav/image/concepts/{concept}", preprocessing)
random_images = get_images_input(f"/Users/martina.gonzales/data/tcav/image/concepts/random_0", preprocessing)

In [38]:
random_images.shape

torch.Size([120, 3, 224, 224])

## Compute activations

Let's load the model:

In [41]:
from torchvision.models import resnet50

model = resnet50(pretrained=True)
model.eval();

Let's create the hooks for the activations:

In [42]:
def get_representation(mod, inp, output):
    output = output.detach()
    features.append(output)

handle = model.layer4.register_forward_hook(get_representation)

Let's obtain the activations and create the feature matrix:

In [63]:
feature_matrix = []

for images_input in [concept_images, random_images]:
    features = []
    out = model(images_input)
    features = torch.cat(features)
    features = features.reshape((features.shape[0], -1))
    feature_matrix.append(features)

feature_matrix = torch.cat(feature_matrix)

In [148]:
feature_matrix[239]

tensor([0.0000, 0.0000, 0.1773,  ..., 0.0000, 0.0000, 0.0000])

The shape of the feature matrix:

In [64]:
feature_matrix.shape

torch.Size([240, 100352])

In [65]:
import numpy as np

class_ids = np.concatenate(
    (np.zeros(len(concept_images)), np.ones(len(random_images))),
    axis=0
)

In [62]:
#import numpy as np
test_2 = torch.cat(feature_matrix)
test_2.shape

torch.Size([240, 2048, 7, 7])

## Compute CAV

Create classifier:

In [173]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

clf = SGDClassifier(alpha=0.01, max_iter=1000, tol=1e-3, random_state=0)
clf = LogisticRegression(C=0.01, random_state=0)

In [219]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_matrix, class_ids, test_size=0.33)

clf.fit(X_train.detach().numpy(), y_train)

preds = torch.tensor(clf.predict(X_test.detach().numpy()))
#return {'accs': (preds == y_test).float().mean()}
score = clf.score(X_test, y_test)

In [229]:
torch.argmax(torch.tensor(preds), dim=1)

/var/folders/d_/2y3g1m_s1kz4x__hr0p6ydx1xnlcbr/T/ipykernel_3281/1463953307.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.argmax(torch.tensor(preds), dim=1)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [175]:
preds.detach().numpy()

array([0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.])

In [176]:
(preds.detach().numpy() == y_test).mean()

1.0

In [177]:
y_test

array([0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.])

In [178]:
cavs = torch.tensor(np.array([-1 * clf.coef_[0], clf.coef_[0]]))

In [179]:
cavs

tensor([[-9.2558e-05, -2.2978e-05, -3.3049e-05,  ..., -1.9948e-04,
         -5.2722e-05, -2.0139e-04],
        [ 9.2558e-05,  2.2978e-05,  3.3049e-05,  ...,  1.9948e-04,
          5.2722e-05,  2.0139e-04]], dtype=torch.float64)

In [180]:
score

1.0

In [181]:
cavs.shape

torch.Size([2, 100352])

## Compute directional derivative